In [5]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
import time
from tqdm import tqdm

GROQ_API_KEY = "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"

def chat_with_llm_model1(user_message, history=None, temperature=0.7):
    
    API_URL = "https://api.groq.com/openai/v1/chat/completions"
    MODEL = "llama3-70b-8192"

    HEADERS = {"Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"}


    if history is None:
        history = []

    history.append({"role": "user", "content": user_message})
    
    payload = {
        "model": MODEL,
        "messages": history,
        "temperature": temperature
    }

    try:
        response = requests.post(API_URL, headers=HEADERS, json=payload)
        response.raise_for_status()
        reply = response.json()["choices"][0]["message"]["content"]
        print(reply)
        return reply
    except requests.exceptions.RequestException as e:
        return f"Request failed: {str(e)}"

OPENROUTER_API_KEY = "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"  # For API - https://openrouter.ai

def chat_with_llm_model2(user_message, history=None, temperature=0.7):
    API_URL = "https://openrouter.ai/api/v1/chat/completions"
    MODEL = "mistralai/mixtral-8x7b-instruct"  

    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://yourdomain.com", 
        "X-Title": "Your App or Project Name"
    }

    if history is None:
        history = []

    history.append({"role": "user", "content":user_message})

    payload = {
        "model": MODEL,
        "messages": history,
        "temperature": temperature,
        "stream": False
    }

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()

        reply = response.json()["choices"][0]["message"]["content"]
        reply = reply.strip()
        print(reply)
        return reply

    except requests.exceptions.RequestException as e:
        return f"Request failed: {str(e)}"

In [6]:
# Load the Excel file and the specific sheet
file_path = r"C:\Users\Udhaya\Desktop\Udhaya DA Files\Power BI (Sir File)\City_Name\city_name.xlsx"
sheet_name = 'Cosmo Input Report'
print("Loading Excel file.")
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Load the prompt from the text file
print("Loading prompt from text file.")
with open(r"C:\Users\Udhaya\Desktop\Udhaya DA Files\Power BI (Sir File)\City_Name\prompt.txt") as file:
    prompt_template = file.read()

# Check for result columns and add them if they don't exist
for col in ['Result_Model1', 'Result_Model2', 'City_Check']:
    if col not in df.columns:
        df[col] = None

Loading Excel file.
Loading prompt from text file.


In [7]:
MAX_THREADS = 5
matches = 0
# Function to process each row and update results for both models
def process_row(index, row):
    main_city = row['Source City name']
    description = row['Description']
    prompt = prompt_template.format(main_city=main_city, description=description)

    result_model1 = chat_with_llm_model1(prompt)
    result_model2 = chat_with_llm_model2(prompt)
    time.sleep(2)
    
    city_name_lower = main_city.lower()
    description_lower = description.lower()
    city_check = (city_name_lower in description_lower) or (f"{city_name_lower}'s" in description_lower)
    
    city_check_result = "City name available and matching" if city_check else "City name not available or mismatched"
    
    return index, result_model1, result_model2, city_check_result

# Parallel processing of the rows
total_rows = len(df)
print("Starting parallel processing of rows.")
with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    futures = {executor.submit(process_row, i, row): i for i, row in df.iterrows()}
    for future in tqdm(futures, total=total_rows, desc="Processing Rows"):
        i, result_model1, result_model2, city_check_result = future.result()
        df.at[i, 'Result_Model1'] = result_model1
        df.at[i, 'Result_Model2'] = result_model2
        df.at[i, 'City_Check'] = city_check_result
    print("finished processing all rows")

def verify_model_results(row):
    expected_result = "a. City name available and main city matching"
    city_check_phrase = "City name available and matching"
    model1_status = "Correct" if row['Result_Model1'] == expected_result else "Result_Model1 is wrong"
    model2_status = "Correct" if row['Result_Model2'] == expected_result else "Result_Model2 is wrong"
    overall_status = ""

    if row['City_Check'] == city_check_phrase:
        if row['Result_Model1'] != expected_result:
            overall_status += model1_status
        if row['Result_Model2'] != expected_result:
            overall_status += " and " + model2_status if overall_status else model2_status
    else:
        # If City_Check is not as expected, no comparison is required with model results.
        overall_status = "City not matching, no need to check models."
    
    return overall_status if overall_status else "Correct"

# Ensure the fourth column exists
if 'Result_Check' not in df.columns:
    df['Result_Check'] = None

print("Verifying model results and updating 'Result_Check' column.")
df['Result_Check'] = df.apply(verify_model_results, axis=1)

# Calculate summary information
correct_count = df['Result_Check'].value_counts().get("Correct", 0)
wrong_model1_count = df['Result_Check'].str.contains("Result_Model1 is wrong").sum()
wrong_model2_count = df['Result_Check'].str.contains("Result_Model2 is wrong").sum()
no_check_needed_count = df['Result_Check'].str.contains("City not matching").sum()

# Output summary information
print(f"Total rows processed: {len(df)}")
print(f"Correct matches: {correct_count}")
print(f"Rows where Result_Model1 is wrong: {wrong_model1_count}")
print(f"Rows where Result_Model2 is wrong: {wrong_model2_count}")
print(f"Rows where model comparison was not needed: {no_check_needed_count}")

# Save the modified DataFrame back to Excel
print("Saving the results to Excel.")
df.to_excel(r"C:\Users\Udhaya\Desktop\Udhaya DA Files\Power BI (Sir File)\City_Name\processed_city_name_2.xlsx", sheet_name=sheet_name, index=False)
print("Processing completed and file saved.")

Starting parallel processing of rows.


Processing Rows:   0%|                                                                          | 0/15 [00:00<?, ?it/s]

a. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching
A. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching


Processing Rows:   7%|████▍                                                             | 1/15 [00:06<01:32,  6.61s/it]

a. City name available and main city matching
A. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching
a. City name available and main city matching


...

a. City name available and main city matching


Processing Rows:  87%|████████████████████████████████████████████████████████▎        | 13/15 [00:15<00:01,  1.13it/s]

a. City name available and main city matching


Processing Rows: 100%|█████████████████████████████████████████████████████████████████| 15/15 [00:18<00:00,  1.26s/it]

finished processing all rows
Verifying model results and updating 'Result_Check' column.
Total rows processed: 15
Correct matches: 13
Rows where Result_Model1 is wrong: 0
Rows where Result_Model2 is wrong: 2
Rows where model comparison was not needed: 0
Saving the results to Excel.
Processing completed and file saved.
